In [1]:
import torch

if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")

In [2]:
device

device(type='cuda')

In [3]:
import mysql.connector

mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="Basketball@1"
)

print(mydb)

In [4]:
cursor = mydb.cursor()

In [5]:
try:
    cursor.execute("CREATE DATABASE CCPD")
except:
    pass

In [6]:
cursor.execute("USE CCPD")

In [7]:
try:
    cursor.execute("CREATE TABLE cars_found (image_name VARCHAR(100), province VARCHAR(5), car_no VARCHAR(10), picture_time VARCHAR(50), detection_time VARCHAR(50))")
except Exception as e:
    print(e)

In [8]:
#encoding:utf-8
import cv2
import torch
from torch.autograd import Variable
import torch.nn as nn
import argparse
import numpy as np
from os import path, mkdir
import os
import time
from load_data import *
from datetime import datetime
from roi_pooling import roi_pooling_ims

def getCarNumber(path):
    ap = argparse.ArgumentParser()
    ap.add_argument("-i", "--input",
                    help="path to the input folder", default = path)
    ap.add_argument("-m", "--model",
                    help="path to the model file", default = 'model_folder/fh02_ccpd.pth')
    ap.add_argument('-f')
    args = vars(ap.parse_args())

    use_gpu = torch.cuda.is_available()
    print (use_gpu)

    numClasses = 4
    numPoints = 4
    imgSize = (480, 480)
    batchSize = 8 if use_gpu else 8
    resume_file = str(args["model"])

    provNum, alphaNum, adNum = 38, 25, 35
    provinces = ["皖", "沪", "津", "渝", "冀", "晋", "蒙", "辽", "吉", "黑", "苏", "浙", "京", "闽", "赣", "鲁", "豫", "鄂", "湘", "粤", "桂",
                 "琼", "川", "贵", "云", "藏", "陕", "甘", "青", "宁", "新", "警", "学", "O"]
    alphabets = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W',
                 'X', 'Y', 'Z', 'O']
    ads = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X',
           'Y', 'Z', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'O']

    class wR2(nn.Module):
        def __init__(self, num_classes=1000):
            super(wR2, self).__init__()
            hidden1 = nn.Sequential(
                nn.Conv2d(in_channels=3, out_channels=48, kernel_size=5, padding=2, stride=2),
                nn.BatchNorm2d(num_features=48),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=2, stride=2, padding=1),
                nn.Dropout(0.2)
            )
            hidden2 = nn.Sequential(
                nn.Conv2d(in_channels=48, out_channels=64, kernel_size=5, padding=2),
                nn.BatchNorm2d(num_features=64),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=2, stride=1, padding=1),
                nn.Dropout(0.2)
            )
            hidden3 = nn.Sequential(
                nn.Conv2d(in_channels=64, out_channels=128, kernel_size=5, padding=2),
                nn.BatchNorm2d(num_features=128),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=2, stride=2, padding=1),
                nn.Dropout(0.2)
            )
            hidden4 = nn.Sequential(
                nn.Conv2d(in_channels=128, out_channels=160, kernel_size=5, padding=2),
                nn.BatchNorm2d(num_features=160),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=2, stride=1, padding=1),
                nn.Dropout(0.2)
            )
            hidden5 = nn.Sequential(
                nn.Conv2d(in_channels=160, out_channels=192, kernel_size=5, padding=2),
                nn.BatchNorm2d(num_features=192),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=2, stride=2, padding=1),
                nn.Dropout(0.2)
            )
            hidden6 = nn.Sequential(
                nn.Conv2d(in_channels=192, out_channels=192, kernel_size=5, padding=2),
                nn.BatchNorm2d(num_features=192),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=2, stride=1, padding=1),
                nn.Dropout(0.2)
            )
            hidden7 = nn.Sequential(
                nn.Conv2d(in_channels=192, out_channels=192, kernel_size=5, padding=2),
                nn.BatchNorm2d(num_features=192),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=2, stride=2, padding=1),
                nn.Dropout(0.2)
            )
            hidden8 = nn.Sequential(
                nn.Conv2d(in_channels=192, out_channels=192, kernel_size=5, padding=2),
                nn.BatchNorm2d(num_features=192),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=2, stride=1, padding=1),
                nn.Dropout(0.2)
            )
            hidden9 = nn.Sequential(
                nn.Conv2d(in_channels=192, out_channels=192, kernel_size=3, padding=1),
                nn.BatchNorm2d(num_features=192),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=2, stride=2, padding=1),
                nn.Dropout(0.2)
            )
            hidden10 = nn.Sequential(
                nn.Conv2d(in_channels=192, out_channels=192, kernel_size=3, padding=1),
                nn.BatchNorm2d(num_features=192),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=2, stride=1, padding=1),
                nn.Dropout(0.2)
            )
            self.features = nn.Sequential(
                hidden1,
                hidden2,
                hidden3,
                hidden4,
                hidden5,
                hidden6,
                hidden7,
                hidden8,
                hidden9,
                hidden10
            )
            self.classifier = nn.Sequential(
                nn.Linear(23232, 100),
                # nn.ReLU(inplace=True),
                nn.Linear(100, 100),
                # nn.ReLU(inplace=True),
                nn.Linear(100, num_classes),
            )

        def forward(self, x):
            x1 = self.features(x)
            x11 = x1.view(x1.size(0), -1)
            x = self.classifier(x11)
            return x


    class fh02(nn.Module):
        def __init__(self, num_points, num_classes, wrPath=None):
            super(fh02, self).__init__()
            self.load_wR2(wrPath)
            self.classifier1 = nn.Sequential(
                # nn.Dropout(),
                nn.Linear(53248, 128),
                # nn.ReLU(inplace=True),
                # nn.Dropout(),
                nn.Linear(128, provNum),
            )
            self.classifier2 = nn.Sequential(
                # nn.Dropout(),
                nn.Linear(53248, 128),
                # nn.ReLU(inplace=True),
                # nn.Dropout(),
                nn.Linear(128, alphaNum),
            )
            self.classifier3 = nn.Sequential(
                # nn.Dropout(),
                nn.Linear(53248, 128),
                # nn.ReLU(inplace=True),
                # nn.Dropout(),
                nn.Linear(128, adNum),
            )
            self.classifier4 = nn.Sequential(
                # nn.Dropout(),
                nn.Linear(53248, 128),
                # nn.ReLU(inplace=True),
                # nn.Dropout(),
                nn.Linear(128, adNum),
            )
            self.classifier5 = nn.Sequential(
                # nn.Dropout(),
                nn.Linear(53248, 128),
                # nn.ReLU(inplace=True),
                # nn.Dropout(),
                nn.Linear(128, adNum),
            )
            self.classifier6 = nn.Sequential(
                # nn.Dropout(),
                nn.Linear(53248, 128),
                # nn.ReLU(inplace=True),
                # nn.Dropout(),
                nn.Linear(128, adNum),
            )
            self.classifier7 = nn.Sequential(
                # nn.Dropout(),
                nn.Linear(53248, 128),
                # nn.ReLU(inplace=True),
                # nn.Dropout(),
                nn.Linear(128, adNum),
            )

        def load_wR2(self, path):
            self.wR2 = wR2(numPoints)
            self.wR2 = torch.nn.DataParallel(self.wR2, device_ids=range(torch.cuda.device_count()))
            if not path is None:
                self.wR2.load_state_dict(torch.load(path))
                # self.wR2 = self.wR2.cuda()
            # for param in self.wR2.parameters():
            #     param.requires_grad = False

        def forward(self, x):
            x0 = self.wR2.module.features[0](x)
            _x1 = self.wR2.module.features[1](x0)
            x2 = self.wR2.module.features[2](_x1)
            _x3 = self.wR2.module.features[3](x2)
            x4 = self.wR2.module.features[4](_x3)
            _x5 = self.wR2.module.features[5](x4)

            x6 = self.wR2.module.features[6](_x5)
            x7 = self.wR2.module.features[7](x6)
            x8 = self.wR2.module.features[8](x7)
            x9 = self.wR2.module.features[9](x8)
            x9 = x9.view(x9.size(0), -1)
            boxLoc = self.wR2.module.classifier(x9)

            h1, w1 = _x1.data.size()[2], _x1.data.size()[3]
            p1 = Variable(torch.FloatTensor([[w1,0,0,0],[0,h1,0,0],[0,0,w1,0],[0,0,0,h1]]).cuda(), requires_grad=False)
            h2, w2 = _x3.data.size()[2], _x3.data.size()[3]
            p2 = Variable(torch.FloatTensor([[w2,0,0,0],[0,h2,0,0],[0,0,w2,0],[0,0,0,h2]]).cuda(), requires_grad=False)
            h3, w3 = _x5.data.size()[2], _x5.data.size()[3]
            p3 = Variable(torch.FloatTensor([[w3,0,0,0],[0,h3,0,0],[0,0,w3,0],[0,0,0,h3]]).cuda(), requires_grad=False)

            # x, y, w, h --> x1, y1, x2, y2
            assert boxLoc.data.size()[1] == 4
            postfix = Variable(torch.FloatTensor([[1,0,1,0],[0,1,0,1],[-0.5,0,0.5,0],[0,-0.5,0,0.5]]).cuda(), requires_grad=False)
            boxNew = boxLoc.mm(postfix).clamp(min=0, max=1)

            # input = Variable(torch.rand(2, 1, 10, 10), requires_grad=True)
            # rois = Variable(torch.LongTensor([[0, 1, 2, 7, 8], [0, 3, 3, 8, 8], [1, 3, 3, 8, 8]]), requires_grad=False)
            roi1 = roi_pooling_ims(_x1, boxNew.mm(p1), size=(16, 8))
            roi2 = roi_pooling_ims(_x3, boxNew.mm(p2), size=(16, 8))
            roi3 = roi_pooling_ims(_x5, boxNew.mm(p3), size=(16, 8))
            rois = torch.cat((roi1, roi2, roi3), 1)

            _rois = rois.view(rois.size(0), -1)

            y0 = self.classifier1(_rois)
            y1 = self.classifier2(_rois)
            y2 = self.classifier3(_rois)
            y3 = self.classifier4(_rois)
            y4 = self.classifier5(_rois)
            y5 = self.classifier6(_rois)
            y6 = self.classifier7(_rois)
            return boxLoc, [y0, y1, y2, y3, y4, y5, y6]


    def isEqual(labelGT, labelP):
        print (labelGT)
        print (labelP)
        compare = [1 if int(labelGT[i]) == int(labelP[i]) else 0 for i in range(7)]
        # print(sum(compare))
        return sum(compare)


    model_conv = fh02(numPoints, numClasses)
    model_conv = torch.nn.DataParallel(model_conv, device_ids=range(torch.cuda.device_count()))
    model_conv.load_state_dict(torch.load(resume_file))
    model_conv = model_conv.cuda()
    model_conv.eval()


    dst = demoTestDataLoader(args["input"].split(','), imgSize)
    trainloader = DataLoader(dst, batch_size=1, shuffle=True, num_workers=1)

    start = time.time()
    for i, (XI, ims) in enumerate(trainloader):

        if use_gpu:
            x = Variable(XI.cuda(0))
        else:
            x = Variable(XI)
        # Forward pass: Compute predicted y by passing x to the model

        fps_pred, y_pred = model_conv(x)

        outputY = [el.data.cpu().numpy().tolist() for el in y_pred]
        labelPred = [t[0].index(max(t[0])) for t in outputY]

        [cx, cy, w, h] = fps_pred.data.cpu().numpy()[0].tolist()

        img = cv2.imread(ims[0])
        
        img_name = ims[0].split('/')[-1]
        print(img_name)
        ti_m = os.path.getmtime(ims[0])
        m_ti = time.ctime(ti_m)
        t_obj = time.strptime(m_ti)
        T_stamp = time.strftime("%Y-%m-%d %H:%M:%S", t_obj)
        
        left_up = [(cx - w/2)*img.shape[1], (cy - h/2)*img.shape[0]]
        right_down = [(cx + w/2)*img.shape[1], (cy + h/2)*img.shape[0]]
        cv2.rectangle(img, (int(left_up[0]), int(left_up[1])), (int(right_down[0]), int(right_down[1])), (0, 0, 255), 2)
        #   The first character is Chinese character, can not be printed normally, thus is omitted.
        lpn = alphabets[labelPred[1]] + ads[labelPred[2]] + ads[labelPred[3]] + ads[labelPred[4]] + ads[labelPred[5]] + ads[labelPred[6]]
        cv2.putText(img, lpn, (int(left_up[0]), int(left_up[1])-20), cv2.FONT_ITALIC, 2, (0, 0, 255))
        cv2.imwrite(ims[0], img)
        cursor.execute("USE CCPD")
        sql = "INSERT INTO cars_found VALUES (%s, %s, %s, %s, %s)"
        val = (img_name, provinces[labelPred[0]], lpn, T_stamp, datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
        cursor.execute(sql, val)

    mydb.commit()

In [9]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import os

def browse_folder():
    global selected_folder_path
    folder_path = filedialog.askdirectory()
    if folder_path:
        selected_folder_path = folder_path
        text_display.delete(1.0, tk.END)  # Clear any previous error messages
        display_image(get_first_image_from_folder(folder_path))

def process_folder():
    if selected_folder_path:
        try:
            getCarNumber(selected_folder_path)
            text_display.delete(1.0, tk.END)  # Clear any previous text
            text_display.insert(tk.END, "Queries added")
        except Exception as e:
            text_display.delete(1.0, tk.END)  # Clear any previous text
            text_display.insert(tk.END, "Error: " + str(e))
    else:
        text_display.delete(1.0, tk.END)
        text_display.insert(tk.END, "Please select a folder first.")

def get_first_image_from_folder(folder_path):
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif')):
            return os.path.join(folder_path, filename)
    return None

def display_image(image_path):
    image = Image.open(image_path)
    image = image.resize((600, 600))  # Resize the image to fit the display
    photo = ImageTk.PhotoImage(image)
    image_label.config(image=photo)
    image_label.image = photo

# Create the main application window
app = tk.Tk()
app.title("Select Folder App")
app.geometry("800x900")  # Set the initial window size

# Global variable to store the selected folder path
selected_folder_path = None

# Create widgets
browse_button = tk.Button(app, text="Browse", command=browse_folder)
ok_button = tk.Button(app, text="OK", command=process_folder)
image_label = tk.Label(app)
text_display = tk.Text(app, wrap=tk.WORD, width=60, height=10)

# Layout widgets
browse_button.pack(pady=20)
image_label.pack(pady=20)
text_display.pack(pady=10)
ok_button.pack(pady=10)

# Start the main event loop
app.mainloop()


True
2.jpg
/home/sc30ash/Documents/Untitled Folder/demo/2.jpg
[0, 0, 17, 24, 26, 26, 2]
4.jpg
/home/sc30ash/Documents/Untitled Folder/demo/4.jpg
[0, 0, 26, 29, 30, 15, 26]
3.jpg
/home/sc30ash/Documents/Untitled Folder/demo/3.jpg
[0, 0, 11, 9, 30, 26, 24]
0.jpg
/home/sc30ash/Documents/Untitled Folder/demo/0.jpg
[0, 0, 25, 32, 29, 21, 33]
1.jpg
/home/sc30ash/Documents/Untitled Folder/demo/1.jpg
[0, 0, 18, 8, 26, 33, 33]


In [10]:
ims

NameError: name 'ims' is not defined

In [ ]:
string = '/home/sc30ash/Documents/Untitled Folder/demo/4.jpg'
string.split('/')